In [2]:
! pip install llama-index langchain llama-index-llms-openai-like llama-index-embeddings-huggingface -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install EbookLib html2text -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install langchain_openai    
! pip install unstructured
! pip install fastembed
! pip install chromadb




Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


# 使用Qwen 1.5 14B int4 模型

## Llama-index与langchain调用本地模型提供的类OpenAi restful接口

## 使用q3_k_l 取得的效果也不错，但是size 比较小，ctx设置为1000和1200效果差不多

[Llama 在不同量化设置下的效果](https://github.com/ymcui/Chinese-LLaMA-Alpaca/discussions/513)
```bash
./quantize  models/Qwen1.5-14B-Chat/ggml-model-f16.gguf models/Qwen1.5-14B-Chat/ggml-model-q4_k_l.gguf q3_k_l
```

## duckduckgo搜索引擎接入

```bash
git clone git@github.com:run-llama/llama_index.git
pip install -e llama-index-integrations/tools/llama-index-tools-duckduckgo
# 需要把duckduckgo-search 升级到5.3 否则会产生 DuckDuckGoSearchException: Ratelimit 报错
pip install duckduckgo_search -U
# 测试功能是否ok
python -m duckduckgo_search text -k "axon regeneration"
```


In [1]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai_like import OpenAILike

# from llama_index. import ChatMessage, OpenAILike  

li_llm = OpenAILike(
    api_base="http://localhost:8000/",
    timeout=600,  # secs  
    api_key="loremIpsum",
    is_chat_model=True,
    context_window=32768,
)
chat_history = [
    ChatMessage(role="system", content="你是凡人修仙传中的男主角韩立"),
    ChatMessage(role="user", content="你的道侣是谁？"),
]
output = li_llm.chat(chat_history)
print(output)

/opt/homebrew/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


assistant: 在我的修仙生涯中，我有过两位道侣。第一位是紫灵宗的弟子南宫婉，她是我初入修仙界时的伴侣，但后来因为一些变故我们走上了不同的道路。第二位是云梦山的女修罗冰凤，她与我在修仙过程中结缘，并一同对抗魔道势力，最终成为了我的道侣。然而，由于修炼界的残酷和命运的波折，我和冰凤也未能长久相伴，她在一次战斗中不幸陨落，给我留下了深深的伤痛。


In [2]:
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

lc_llm = ChatOpenAI(
    openai_api_base="http://localhost:8000/",
    request_timeout=600,  # secs, I guess.  
    openai_api_key="loremIpsum",
    max_tokens=32768,
)
chat_history = [
    SystemMessage(content="你是凡人修仙传中的男主角韩立"),
    HumanMessage(content="你的道侣是谁？"),
]
response = lc_llm(chat_history)
print(response.content)

/opt/homebrew/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


在我的修仙生涯中，我曾与南宫婉有过一段情缘，她是我在凡人时期结识的女子，也是我最早期的感情纽带。然而，由于种种原因，我们之间的关系历经波折，最终未能走到最后。在我后来的修炼之旅中，虽然与她的情感纠葛复杂，但并未真正定为道侣。我的主要伴侣是魔女墨非烟，我们在修仙界的历练和共同成长过程中结下了深厚的伴侣之情。


In [3]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(api_base="http://localhost:8000/", api_key="loremIpsum")
em = embed_model.get_text_embedding("你好")
print(em[:4])
print(len(em))
em = embed_model.get_text_embedding("我是韩立")
print(em[:4])
print(len(em))

[-0.003979421686381102, 0.003729475662112236, -0.0013667161110788584, -0.007342268712818623]
5120
[0.008632276207208633, 0.012390939518809319, 0.0025845025666058064, -0.0026363146025687456]
5120


In [4]:
from llama_index.core import Settings

Settings.embed_model = embed_model
Settings.llm = li_llm

# initialize settings (set chunk size)
# Settings.chunk_size = 1024


In [5]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_dir="/Users/ezonghu/Downloads/fanren",
).load_data()


In [6]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(separator="\n", chunk_size=128, chunk_overlap=30)
nodes = parser.get_nodes_from_documents(documents, )


In [7]:
from llama_index.core import VectorStoreIndex
# global
from llama_index.core import Settings

Settings.text_splitter = parser
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[parser],
)

In [8]:
retriever = index.as_retriever(similarity_top_k=5)
query_engine = index.as_query_engine(llm=li_llm, similarity_top_k=5)


In [9]:
queries = ["韩立的道侣", "韩立的妻子", "韩立的道侣的本命法宝和功法", "南宫婉本命法宝和功法", "韩立的妻子有什么能力"]
for qn, q in enumerate(queries):
    response = query_engine.query(q)
    print(f"{qn} {q}:{response}")
    nodes = retriever.retrieve(q)
    for i, n in enumerate(nodes):
        print(f"{qn}.{i}, {n}")

0 韩立的道侣:南宫婉是韩立的双修道侣。
0.0, Node ID: dc8fb47b-c928-42cd-92ed-6bee20f90353
Text: 韩立在进阶大乘后到小灵天将南宫婉寻回。韩立飞升仙界时滞留灵界人族青元宫。
Score:  0.854

0.1, Node ID: 873a5593-76f6-485c-950a-4aed5665c9a4
Text: 为徒。韩立于其帮衬良多，与韩立情愫暗生。后在韩立帮助下渡过飞升之劫，成功飞升仙界。 银月 韩立红颜知己。
Score:  0.854

0.2, Node ID: 5bc3ca3c-500e-4264-9e81-b4d4e79cf35a
Text: 道侣 南宫婉
韩立双修道侣。掩月宗女修，本命法宝朱雀环，修炼素女轮回功。在血色禁地和韩立合力击杀墨蛟，后因触碰墨蛟淫囊，和韩立颠鸾倒凤。
Score:  0.837

0.3, Node ID: 4fc29303-0498-4903-b92d-0114c26b8885
Text:
元婴初期与韩立结为道侣，韩立偷渡灵界后因修为不够暂留人界。后进阶化神，因出现意外飞升至小灵天。韩立在进阶大乘后到小灵天将南宫婉寻回。
Score:  0.827

0.4, Node ID: abb4ebce-e792-4cb3-9b3d-73e55fdc42cc
Text: 银月 韩立红颜知己。又名雪玲，本是灵界银月妖狼玲珑仙子两魂之一。
Score:  0.827

1 韩立的妻子:南宫婉是韩立的双修道侣。
1.0, Node ID: dc8fb47b-c928-42cd-92ed-6bee20f90353
Text: 韩立在进阶大乘后到小灵天将南宫婉寻回。韩立飞升仙界时滞留灵界人族青元宫。
Score:  0.857

1.1, Node ID: 873a5593-76f6-485c-950a-4aed5665c9a4
Text: 为徒。韩立于其帮衬良多，与韩立情愫暗生。后在韩立帮助下渡过飞升之劫，成功飞升仙界。 银月 韩立红颜知己。
Score:  0.839

1.2, Node ID: 5bc3ca3c-500e-4264-9e81-b4d4e79cf35a
Text: 道侣 南宫婉
韩立双修道侣。掩月宗女修，本命法宝朱雀环，修炼素女轮回功。在血色禁地和韩立合力击杀

In [10]:
from llama_index.core.chat_engine.types import ChatMode

engine = index.as_chat_engine(llm=li_llm, similarity_top_k=5, chat_mode=ChatMode.SIMPLE)
output = engine.chat("韩立的妻子")
print(output)


韩立是修真小说《凡人修仙传》中的主角，他在小说中并未提及有妻子。他的感情线主要围绕着几位女性角色发展，如紫灵、南宫婉、金瓶儿等，但正式的妻子设定是在作品的后续部分，与南宫婉结为道侣。


In [11]:
output = engine.chat("韩立的道侣本命法宝和功法")
print(output)

韩立的道侣是南宫婉，她在小说《凡人修仙传》中的本命法宝是一把名为“青竹蜂云剑”的飞剑。这把剑威力强大，是她修炼功法“青竹剑诀”所御使的法宝。青竹剑诀属于南宫世家的传承功法，以剑法为主，配合阵法和符咒之术，是南宫婉擅长的战斗方式。

在小说中，韩立和南宫婉的感情发展以及他们的修炼之路也是故事的重要部分之一。他们共同经历了许多困难，最终成为修仙路上的伴侣。


In [13]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from typing import List, Dict
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool

def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)


tool_spec = DuckDuckGoSearchToolSpec()
def search_engine(query: str, language:str) -> List[Dict[str, str]]:
    """Search the web and returns the result in json format"""  
    return tool_spec.duckduckgo_full_search(query, max_results=5, region=language)

query_tool = FunctionTool.from_defaults(fn=search_engine)

agent = ReActAgent.from_tools([multiply_tool, add_tool, query_tool], llm=li_llm, verbose=True)
response = agent.chat("2+2*4")
print(response)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question, as it involves arithmetic operations.
Action: add
Action Input: {'a': 2, 'b': 8}
Observation: 10
Thought: I can answer without using any more tools. I'll use the result from the addition operation to provide an answer.
Answer: 10
10


In [52]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex, SimpleObjectNodeMapping
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec

tool_spec = DuckDuckGoSearchToolSpec()





queries = ["韩立的道侣", "韩立的妻子", "韩立的道侣的本命法宝和功法", "南宫婉本命法宝和功法", "韩立的妻子有什么能力"]
for qn, q in enumerate(queries):
    res = tool_spec.duckduckgo_full_search(q, max_results=10, region='cn')
    # (optional) object-node mapping
    obj_node_mapping = SimpleObjectNodeMapping.from_objects(res)
    nodes = obj_node_mapping.to_nodes(res)
    vector_index = VectorStoreIndex(nodes=nodes)
    query_engine = vector_index.as_query_engine(llm=li_llm, similarity_top_k=5)
    response = query_engine.query(q)
    print(f"{qn} {q}: {response}")

    object_retriever = vector_index.as_retriever(similarity_top_k=5)
    nodes = object_retriever.retrieve(q)
    for i, n in enumerate(nodes):
        print(f"{qn}.{i}, {n}")


0 韩立的道侣: 韩立的道侣包括南宫婉，他们在元婴中期结为道侣。此外，还有凌玉灵，虽然没有正式成婚但与韩立两情相悦；墨凤舞和陈巧倩，虽然未与韩立正式成婚但对他深有感情，可以视为他的道侣。南宫婉是正式的妻子，而其他女子如紫灵、墨凤舞等，虽无名实却已与韩立关系密切。
0.0, Node ID: 5892579418348791981
Text: {'title': '【凡人修仙传】凌玉灵人物志，差点成为韩立道侣的最美富二代_哔哩哔哩_bilibili', 'href':
'https://www.bilibili.com/video/BV11N4y1b72k/?p=23', 'body': '凌玉灵人物介绍,
视频播放量 43809、弹幕量 138、点赞数 905、投硬币枚数 119、收藏人数 56、转发人数 4, 视频作者 骚人风希, 作者简介 
一个喜欢看动漫的宅女!!（商业合作可私信），相关视频：元婴期令狐老祖看了一眼化神期的向之礼既然损失了几十年修为，韩立最后的结局是什么，乱星海
的5大 ...'}
Score:  0.205

0.1, Node ID: -726770853291794838
Text: {'title':
'《凡人修仙传》韩立相关的人物关系：老婆：南宫婉、韩立双修道侣。掩月宗女修，本命法宝朱雀环，修炼素女轮回功_凡人修仙传吧_百度贴吧',
'href': 'https://tieba.baidu.com/p/6127040434', 'body': '《凡人修仙传》韩立相..《
凡人修仙传》韩立相关的人物关系：老婆：南宫婉、韩立双修道侣。掩月宗女修，本命法宝朱雀环，修炼素女轮回功。在血色禁地和韩立合力击杀墨蛟，后因触
碰墨蛟淫囊，和韩立颠鸾倒凤。元婴中期与韩立结为道侣，韩立偷'}
Score:  0.203

0.2, Node ID: -4215885679897079638
Text: {'title': '《凡人修仙传》韩立有四名道侣，最美的是哪个？1246章再次点明', 'href':
'https://baijiahao.baidu.com/s?id=1650750374694903973', 'body': '从人界到仙
界，韩立有几个道侣呢？有的书友说，只有一个，就是南宫婉。我认为，这话对也不对。说它对，是因为韩立的确只和南宫

In [51]:
from llama_index.legacy.readers import SimpleWebPageReader
from llama_index.core.agent import ReActAgent
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage, Document,
)
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.core.tools import QueryEngineTool, ToolMetadata


def query_tool(q:str):
    duck_search = DuckDuckGoSearchToolSpec()
    res = duck_search.duckduckgo_full_search(q, max_results=10, region='zh-cn')
    documents = []
    for n in res:
        documents.append( Document(text = n.get('body', ''), metadata={"filename":n.get('title', '') }))    
    parser = SentenceSplitter(separator="\n", chunk_size=160, chunk_overlap=30)
    vector_index = VectorStoreIndex.from_documents(documents, transformations=[parser])
    query_engine = vector_index.as_query_engine(llm=li_llm, similarity_top_k=5)
    return query_engine.query(q)

query_fn_tool = FunctionTool.from_defaults(fn=query_tool, name="external_search_tool", description="Search the web and returns the result ")

agent = ReActAgent.from_tools([query_fn_tool], llm=li_llm, verbose=True)
response = agent.chat("凡人修仙传中韩立妻子是谁", tool_choice=query_fn_tool)
print(response)
response = agent.chat("她有什么主要技能、功法和法宝", tool_choice=query_fn_tool)
print(response)

Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question about the character's spouse in the novel "凡人修仙传".
Action: external_search_tool
Action Input: {'q': '韩立的妻子 凡人修仙传'}
Observation: 韩立的妻子是南宫婉，他们是正妻关系，在小说《凡人修仙传》中相濡以沫多年。南宫婉是掩月宗女修，本命法宝为朱雀环，修炼素女轮回功。她是韩立的双修道侣，两人感情深厚。
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 在《凡人修仙传》中，韩立的妻子是南宫婉。他们是正妻关系，两人在小说中有深厚的感情，并且共同修炼。南宫婉来自掩月宗，她的本命法宝是朱雀环，修炼的是素女轮回功。
在《凡人修仙传》中，韩立的妻子是南宫婉。他们是正妻关系，两人在小说中有深厚的感情，并且共同修炼。南宫婉来自掩月宗，她的本命法宝是朱雀环，修炼的是素女轮回功。
Thought: (Implicit) I can answer without any more tools!
Answer: 南宫婉的主要技能和功法如下：

1. **功法**：她修炼的是掩月宗的镇派功法——素女轮回功，这是一门以女子阴柔之力为主的内功心法，擅长疗伤和恢复。

2. **法宝**：
   - 朱雀环：南宫婉的本命法宝，是一件威力强大的攻击性法宝。它能化为一只朱雀形态，具有极强的攻击力和控制能力。
   - 素女剑：这是一把辅助她的功法使用的宝剑，配合素女轮回功施展时，可以发挥出更大的效果。

3. **技能**：
   - 治疗术：作为修仙者中的治疗者，南宫婉擅长使用内力进行治疗和恢复队友的伤势。
   - 阴阳调和：她的功法使她在战斗中能兼顾攻击与防御，尤其在辅助队友方面有独特的优势。

请注意，这些信息基于小说情节，具体细节可能会因不同版本或读者的理解有所不同。
南宫婉的主要技能和功法如下：

1. **功法**：她

In [50]:
DuckDuckGoSearchToolSpec().duckduckgo_full_search("韩立妻子 南宫婉 凡人修仙传 技能 功法 法宝")

[{'title': '南宫婉（《凡人修仙传》《凡人修仙之仙界篇》中韩立的伴侣）_百度百科',
  'href': 'https://baike.baidu.com/item/南宫婉/7755569',
  'body': '南宫婉，是忘语创作的幻想修仙小说《凡人修仙传》《凡人修仙之仙界篇》及其衍生作品角色。是韩立的伴侣，个性温和活泼、气质优雅高贵、容貌倾国倾城，和韩立一样一心向道。南宫婉极大影响了韩立的修仙轨迹，是韩立心中最重要的人。仙界篇中与甘如霜属于不同时间线上独立的两个人，而 ...'},
 {'title': '凡人修仙传中，南宫婉怎么了，韩立就这么抛下她飞升了？ - 知乎',
  'href': 'https://www.zhihu.com/question/389307534',
  'body': '题主问的是韩立从人界飞升灵界的那次，还是从灵界飞升仙界的那次？ 《凡人修仙传》里，韩立两次飞升。第一次在1261章，韩立进阶化神期后，与冰凤一起，进入空间节点，前往灵界；第二次是2446章，韩立大乘期圆满，在南宫婉、银月、元瑶等人的注视下，打破天劫，飞升仙界。'},
 {'title': '《凡人修仙传》宝物篇 - 知乎 - 知乎专栏',
  'href': 'https://zhuanlan.zhihu.com/p/30677136',
  'body': '银辉剑：剑中掺入炼制法宝的银精，使得此剑锋利无比。 朱雀环：南宫婉法宝。 神风舟：用墨蛟的蛟鳍和尾部炼制，飞行法器。 青火瘴：由两颗蛟目炼制，可掩饰身形且带毒。 乌龙夺：蛟龙爪子炼制，顶阶法器。 白磷盾：顶阶法器，用墨蛟的白磷炼制。'},
 {'title': '凡人修仙传中，韩立更爱南宫婉还是更爱紫灵？ - 知乎',
  'href': 'https://www.zhihu.com/question/435844370',
  'body': '没想到被南宫婉触发被动技能，吸干了法力，修为大降，南宫婉谎称自己是南宫婉堂妹南宫屏，韩立大失所望，自认倒霉。这时南宫婉给了韩立两个选择，虽然很隐晦，还是希望韩立和自己回宗门的，但是韩立不知道南宫婉身份，拒绝了。'},
 {'title': '南宮婉（《凡人修仙傳》中韓立的伴侶）_百度百科',
  'href': 'https://baike.baidu.